In [2]:
import pandas as pd
import numpy as np
from datetime import datetime,timedelta

In [5]:
df_customers = pd.read_csv("..\data\olist_customers_dataset.csv")
print("df_customers", df_customers.shape)
df_sellers = pd.read_csv("..\data\olist_sellers_dataset.csv")
print("df_sellers", df_sellers.shape)
df_orders = pd.read_csv("..\data\olist_orders_dataset.csv")
print("df_orders", df_orders.shape)
df_products = pd.read_csv("..\data\olist_products_dataset.csv")
print("df_products", df_products.shape)
df_orderItems = pd.read_csv("..\data\olist_order_items_dataset.csv")
print("df_orderItems", df_orderItems.shape)
df_orderReviews = pd.read_csv("..\data\olist_order_reviews_dataset.csv")
print("df_orderReviews", df_orderReviews.shape)
df_orderPayments = pd.read_csv("..\data\olist_order_payments_dataset.csv")
print("df_orderPayments", df_orderPayments.shape)
df_geolocation = pd.read_csv("..\data\olist_geolocation_dataset.csv")
print("df_geolocation", df_geolocation.shape)
df_productCategory = pd.read_csv("..\data\product_category_name_translation.csv")
print("df_productCategory", df_productCategory.shape)


df_products_english = pd.merge(df_products,df_productCategory ,on="product_category_name",how="left")
print("df_products_english", df_products_english.shape)


df_customers (99441, 5)
df_sellers (3095, 4)
df_orders (99441, 8)
df_products (32951, 9)
df_orderItems (112650, 7)
df_orderReviews (100000, 7)
df_orderPayments (103886, 5)
df_geolocation (1000163, 5)
df_productCategory (71, 2)
df_products_english (32951, 10)


In [153]:
def getDateTime(date_str):
    return datetime.strptime(date_str, '%Y-%m-%d %H:%M:%S')

In [154]:
def CheckIfNullExistsInDataframe(df):
    return df.isnull().values.any()

def GetCountOfNullValuesInDataframe(df):
    return df.isnull().sum()

In [155]:
def Merge_Orders_and_Customers(df_orders,df_customers):
    return pd.merge(df_orders,df_customers,on="customer_unique_id",how="left")

def Merge_OrderItemsSellers_and_Orders(df_orderitems_sellers,df_orders):
    return pd.merge(df_orderitems_sellers,df_orders[["order_id","order_status","order_purchase_timestamp","order_approved_at","order_delivered_carrier_date","order_delivered_customer_date"]],on="order_id",how="left")

def Merge_OrdersItems_and_Sellers(df_orderItems,df_sellers):    
    return  pd.merge(df_orderItems[["order_id","order_item_id","seller_id"]],df_sellers[["seller_id"]],on="seller_id",how="left")


In [156]:
def GetCustomerLatestOrderDate(df_orders_customers,df_customers):
    df_customers['latest_order_date'] = df_orders_customers.groupby("customer_unique_id")["order_purchase_timestamp"].transform(max)
    df_customers['latest_order_date'] = pd.to_datetime(df_customers['latest_order_date'])
    return df_customers

def GetSellerLatestOrderDate(df_orders_orderitems_sellers,df_sellers):
    df_sellers['latest_order_date'] = df_orders_orderitems_sellers.groupby("seller_id")["order_approved_at"].transform(max)
    df_sellers['latest_order_date'] = pd.to_datetime(df_sellers['latest_order_date'])
    return df_sellers


In [157]:
def SetChurnUsers(df,checkColumn,churn_month_duration):
    # lets take the latest order date (2018-10-17 17:30:18) as the current date and 
    # call user a churn user if he/she hasnt placed an order for last 3 months
    # churn_user = True when user hasnt ordered for last 3 months
    se_date = GetMaxColumnValue(df,checkColumn)
    print("Max date which is used to check if user is churn or not = ",se_date)
    df["churn_user"] = True
    df.loc[ df[checkColumn] >= ( se_date - timedelta( days = (churn_month_duration*30.42))), "churn_user"] = False
    
    return df

In [158]:
def GetRowsWhereColumnValueEquals(df,columnName,value):
    return df.loc[df[columnName] == value]

def GetMaxColumnValue(df,columnName):
    return df[columnName].max()

def GetMinColumnValue(df,columnName):
    return df[columnName].min()

def GetUnqiueColumnValues(df,columnName):
    return df[columnName].unique()

def GetUnqiueColumnValuesLength(df,columnName):
    return len(df[columnName].unique())

def GetCountOfGroupBy(df,GroupbyOn,ColumnCountOn):
    return df.groupby(GroupbyOn)[ColumnCountOn].count()

In [159]:
df_orders_customers =  Merge_Orders_and_Customers(df_orders,df_customers)
df_customers = GetCustomerLatestOrderDate(df_orders_customers,df_customers)
df_customers = SetChurnUsers(df=df_customers,checkColumn="latest_order_date",churn_month_duration = 3)

KeyError: 'customer_unique_id'

In [160]:
NumberOfCustomersInCity = GetCountOfGroupBy(df_customers,"customer_city","customer_unique_id")
MaxCustomerCity = NumberOfCustomersInCity.idxmax()
MaxCustomerCityValue = NumberOfCustomersInCity.max()

MaxCustomerCity,MaxCustomerCityValue

('sao paulo', 15540)

In [161]:
df_orderitems_sellers = Merge_OrdersItems_and_Sellers(df_orderItems,df_sellers)
df_orders_orderitems_sellers = Merge_OrderItemsSellers_and_Orders(df_orderitems_sellers,df_orders)

print("Does any null value exist? ","Yes" if CheckIfNullExistsInDataframe(df_orders_orderitems_sellers) == True else "No")
print(GetCountOfNullValuesInDataframe(df_orders_orderitems_sellers))
df_orders_orderitems_sellers.shape

#null exists because there are some orders whose order status is cancelled

Does any null value exist?  Yes
order_id                            0
order_item_id                       0
seller_id                           0
order_status                        0
order_purchase_timestamp            0
order_approved_at                  15
order_delivered_carrier_date     1194
order_delivered_customer_date    2454
dtype: int64


(112650, 8)

In [162]:
GetUnqiueColumnValues(df_orders_orderitems_sellers,"order_status")

array(['delivered', 'shipped', 'canceled', 'invoiced', 'processing',
       'approved', 'unavailable'], dtype=object)

In [163]:
df_orders_orderitems_sellers.head()

,order_id,order_item_id,seller_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date
0,00010242fe8c5a6d1ba2dd792cb16214,1,48436dade18ac8b2bce089ec2a041202,delivered,2017-09-13 08:59:02,2017-09-13 09:45:35,2017-09-19 18:34:16,2017-09-20 23:43:48
1,00018f77f2f0320c557190d7a144bdd3,1,dd7ddc04e1b6c2c614352b383efe2d36,delivered,2017-04-26 10:53:06,2017-04-26 11:05:13,2017-05-04 14:35:00,2017-05-12 16:04:24
2,000229ec398224ef6ca0657da4fc703e,1,5b51032eddd242adc84c38acab88f23d,delivered,2018-01-14 14:33:31,2018-01-14 14:48:30,2018-01-16 12:36:48,2018-01-22 13:19:16
3,00024acbcdf0a6daa1e931b038114c75,1,9d7a1d34a5052409006425275ba1c2b4,delivered,2018-08-08 10:00:35,2018-08-08 10:10:18,2018-08-10 13:28:00,2018-08-14 13:32:39
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,df560393f3a51e74553ab94004ba5c87,delivered,2017-02-04 13:57:51,2017-02-04 14:10:13,2017-02-16 09:46:09,2017-03-01 16:42:31


In [164]:
df_orders_orderitems_sellers.iloc[:, 4:] = df_orders_orderitems_sellers.iloc[:, 4:].apply(pd.to_datetime, errors='coerce')

df_sellers =  GetSellerLatestOrderDate(df_orders_orderitems_sellers,df_sellers)

df_sellers = SetChurnUsers(df=df_sellers,checkColumn="latest_order_date",churn_month_duration=3)

Max date which is used to check if user is churn or not =  2018-09-03 17:40:06


In [180]:
print(df_sellers.shape)
df_sellers.head()

(3095, 6)


,seller_id,seller_zip_code_prefix,seller_city,seller_state,latest_order_date,churn_user
0,3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP,2018-08-18 21:30:14,False
1,d1b65fc7debc3361ea86b5f14c68d2e2,13844,mogi guacu,SP,2018-07-25 16:45:18,False
2,ce3ad9de960102d0677a81f5d0bb7b2d,20031,rio de janeiro,RJ,2018-06-16 21:55:56,False
3,c0f3eea2e14555b6faeea3dd58c1b1c3,4195,sao paulo,SP,2018-08-20 20:10:27,False
4,51a04a8a6bdcb23deccc82b0b80742cf,12914,braganca paulista,SP,2018-03-10 21:35:22,True


In [188]:
df_orders_customers.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,latest_order_date,churn_user
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,7c396fd4830fd04220f754e42b4e5bff,3149,sao paulo,SP,2018-07-01 07:36:35,True
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00,af07308b275d755c9edb36a90c618231,47813,barreiras,BA,2017-12-15 16:39:14,True
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00,3a653a41f6f9fc3d2a113cf8398680e8,75265,vianopolis,GO,2017-11-16 16:20:02,True
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00,7c142cf63193a1473d2e66489a9ae977,59296,sao goncalo do amarante,RN,2017-12-07 22:50:27,True
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00,72632f0f9dd73dfee390c9b22eb56dd6,9195,santo andre,SP,2017-10-14 18:29:57,True
